# Question Data Tutorial
The goal of this tutorial is to introduce some useful functions and show how to do typical tasks when working with quantitative educational data. This tutorial assumes you already have basic knowledge of Python and Pandas and have already completed the Exam data tutorial and/or mastered the skills in that lesson.

In this lesson, you will learn the following:
* How recode variables to new values
* Change the names of data frame columns
* Wide vs long format for data frames
* Concatenate and merge data frames

***
Created by Dr. Nicholas Young

Last modified: April 3, 2025

Python version: 3.11.9

As will likely be the case for most your files, we start by importing numpy, pandas, and pyplot

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Now let's read in our question files. This time we have two files, one from a morning section of a course and one from an afternoon section of a course. To minimize the amount of typing, I'll use 'm' for morning and 'a' for afternoon in the variables I store my data.

Some background on this data:
* This instructor's exam had 4 multiple choice questions and 2 free response questions. We only have the multiple choice questions.
* 1 is correct while 0 is incorrect
* Q1 and Q4 are the same on both exams but the instructor swapped the order of Q2 and Q3 on the two exams. This means that Q2 on the morning exam is Q3 on the afternoon exam and Q2 on the morning exam is Q3 on the afternoon exam.
* The instructor also collected some data about the students taking their exam. This is stored in the `class_demographics.csv` file.

In [2]:
m_data = pd.read_csv('morning_exam.csv')
a_data = pd.read_csv('afternoon_exam.csv')

Let's start by inspecting our data and see what we are working with

In [3]:
m_data.head()

,id,section,Q1,Q2,Q3,Q4
0,100,morning,0,1,0,0
1,101,morning,0,1,1,0
2,102,morning,0,0,0,0
3,103,morning,0,1,1,0
4,104,morning,1,0,1,1


In [7]:
a_data.head()

,id,section,Q1,Q2,Q3,Q4
0,200,afternoon,0,0,B,0
1,201,afternoon,1,0,C,0
2,202,afternoon,1,1,B,0
3,203,afternoon,0,0,B,0
4,204,afternoon,0,1,D,0


You'll likely notice a problem in that Q3 is the students' responses rather than whether they are correct or not. Let's address that first.

## Changing the values of variables based on a condition.
As is often the case, our data isn't exactly the way we want it. Here, we have a column with student answers rather than whether it is correct not. We want to convert that to correct or not (1/0).

Let us assume that "B" is the correct answer to Q3 on the afternoon exam.

There are two ways to do this. Which one we want to use depends on how many different values there will be after the conversion. For a binary outcome like correct or not, we can use a logic statement. Here, we ask if Q3 is equal to B. If so, assign a 1 and otherwise, assign a zero.

In [9]:
a_data['Q3'] =(a_data['Q3'] == "B").astype(int)

Here the `as.type(int)` is important because when we test whether each response in Q3 is B, we get a list of True and False back. True is equivalent to 1 and False is equivalent to 0 so we can convert the logic variable to an integer to get ones and zeros.

If we had multiple values were assigning to (say you were trying to categorize the alternative conception that each response targets), you could use `replace`. In `replace`, you provide a dictionary with the current values followed by the new values. This provides much more flexibility than the previous way.

In [11]:
a_data['Q3'] = a_data['Q3'].replace({"A": 0, "B": 1, "C": 0, "D": 0})

If we look at the afternoon data now, we will see that all of the questions are now in correct/incorrect format

In [13]:
a_data.head()

,id,section,Q1,Q2,Q3,Q4
0,200,afternoon,0,0,1,0
1,201,afternoon,1,0,0,0
2,202,afternoon,1,1,1,0
3,203,afternoon,0,0,1,0
4,204,afternoon,0,1,0,0


## Examining performance

Now that the data are in the correct format, let's see on how the students did. We want to find the fraction of responses that are correct per question as well as the student's score.

Try this on your own in the cells below. If you are stuck, expand the header below called solution to see what I found or look back to the Exam data tutorial for useful functions.

In [ ]:
# use this block to put your code for the fraction of correct responses per question
# as a hint, 55% of students answered Q2 correctly in the morning class.



In [ ]:
# use this block to put your code for total score per student
# as a hint, student 101 earned a score of 2.



### Examining performance solution
Try to do this yourself before looking at the solution.

In [ ]:
m_data[['Q1', 'Q2', 'Q3', 'Q4']].mean()

In [ ]:
a_data[['Q1', 'Q2', 'Q3', 'Q4']].mean()

Because the responses are stored as correct or not, the number of correct responses is sum of the column and the number of attempts is the number of rows in the column. This is just the average so I can find the fraction of correct responses by taking the average of each column.

In [ ]:
m_data['score'] = m_data['Q1'] + m_data['Q2'] + m_data['Q3'] + m_data['Q4']
a_data['score'] = a_data['Q1'] + a_data['Q2'] + a_data['Q3'] + a_data['Q4']

In [ ]:
m_data.head()

Again, since each question is graded correct or not, each student's score is the number of questions they get correct, which is the sum of the four question columns.

## Combining data sets
So far, we've treated the two data sets as separate things. However, as the questions are the same, just in a different order, it might be useful to combine the data to get a better sense of how students are going on these exams. For example, we might need to collect data from multiple classes in order to get a sufficient sample size.

Let's introduce some techniques that could be useful for doing this.

If I want to combine the data together into a single data frame with 200 rows instead of second data frames with 100 rows, I need each data frame to have the same column names.

We've already met this condition so we can use the `concat` function. I include `ignore_index=True` so that a new index will be created. Otherwise, the individual indices of the existing data frames will be kept and you would have two rows with each index (0-99) because `a_data` and `m_data` each have an index of 0-99.

In [15]:
pd.concat([a_data, m_data], ignore_index=True)

,id,section,Q1,Q2,Q3,Q4
0,200,afternoon,0,0,1,0
1,201,afternoon,1,0,0,0
2,202,afternoon,1,1,1,0
3,203,afternoon,0,0,1,0
4,204,afternoon,0,1,0,0
...,...,...,...,...,...,...
195,195,morning,1,1,1,0
196,196,morning,0,0,0,1
197,197,morning,1,0,0,1
198,198,morning,1,1,1,0


Here's what that looks like without the `ignore_index=True` part. Notice that the first column ends at 99 now instead of 199. Generally it's a good idea to not have the same index twice.

In [17]:
pd.concat([a_data, m_data])

,id,section,Q1,Q2,Q3,Q4
0,200,afternoon,0,0,1,0
1,201,afternoon,1,0,0,0
2,202,afternoon,1,1,1,0
3,203,afternoon,0,0,1,0
4,204,afternoon,0,1,0,0
...,...,...,...,...,...,...
95,195,morning,1,1,1,0
96,196,morning,0,0,0,1
97,197,morning,1,0,0,1
98,198,morning,1,1,1,0


Now that these are combined, it should be easy to do what we did before and take the column means, right? Well, not quite. If we took the column means, this would still work and not throw an error but the means wouldn't mean what we think they do. Because Q2 and Q3 are not the same on each exam, if we took the mean of the columns, we would answer the question of how did students do on the second and third question on each exam rather than how did student do on Q2 and Q3. (Make sure you understand the difference).

There are a few ways to proceed. One of the easiest ways is to just rename and reorder the columns so that the same questions are in columns with the same name and same location. To prevent confusion, I'm going to use "item" to refer to the specific question and question to refer to the order of it on the exam.

So for the morning exam, we have
* Q1 = I1
* Q2 = I2
* Q3 = I3
* Q4 = I4

and for the afternoon exam, we have
* Q1 = I1
* Q2 = I3
* Q3 = I2
* Q4 = I4

Let's rename the columns using this scheme.

In [24]:
a_data = a_data.rename(columns = {'Q1':'I1', 'Q2':'I2', 'Q3':'I3', 'Q4':'I4'})
m_data = m_data.rename(columns = {'Q1':'I1', 'Q3':'I2', 'Q2':'I3', 'Q4':'I4'})

First, let's see what this did to the dataframes

In [26]:
a_data.head()

,id,section,I1,I2,I3,I4
0,200,afternoon,0,0,1,0
1,201,afternoon,1,0,0,0
2,202,afternoon,1,1,1,0
3,203,afternoon,0,0,1,0
4,204,afternoon,0,1,0,0


In [29]:
m_data.head()

,id,section,I1,I3,I2,I4
0,100,morning,0,1,0,0
1,101,morning,0,1,1,0
2,102,morning,0,0,0,0
3,103,morning,0,1,1,0
4,104,morning,1,0,1,1


Python is smart enough to match columns if they are in different orders as long as the names are the same. If we `concat` now, we should get what we are hoping for.

In [38]:
am_data = pd.concat([a_data, m_data], ignore_index=True)
am_data.head()

,id,section,I1,I2,I3,I4
0,200,afternoon,0,0,1,0
1,201,afternoon,1,0,0,0
2,202,afternoon,1,1,1,0
3,203,afternoon,0,0,1,0
4,204,afternoon,0,1,0,0


Now we can get the performance on each item

In [41]:
am_data[['I1', 'I2', 'I3', 'I4']].mean()

I1    0.485
I2    0.695
I3    0.525
I4    0.190
dtype: float64

We see that students did best on item 2 and the worst on item 4. They did about the same on items 1 and 3.

## Switching between long and wide format
There are two common formats to store data, the *wide* format and the *long* format. In the wide format, each variable has its own column while in long format, each observation has its own row, with the variable and its value as separate columns. So far, we have only worked in the wide format. Here, we will show how to convert between the two formats.

Wide format is probably how you will be working most of the time. However, long format can be useful depending on the task. If you have a lot of blank data (say there are four versions of the exam each with 10 questions and all of the versions are completely different from each other so each student only answers 10 of the 40 possible questions), long format can be a more efficient way to store data. That is, long format can also result in smaller file sizes if the long format equivlent would include a lot of missing data. This can be important if you are working with multi-GB size files or real-world data that is being added to in real-time.

Let's show how to convert between these and then an example of where long format can help.

To move from wide format to long format, we use the `melt` function. We need specify a few things to switch from wide to long. First, we need to identify the columns we want to remain as columns in the `id_vars` parameter. Next, we need to identify the columns we want to convert to a single column in the `value_vars` parameter. We specify the name of this new column in the `var_name` parameter. The values of the variables we listed in `value_vars` will be stored in a new columm whose name we specify in `value_name`.

For the am_data, we want to keep the `id` and `section` variables as columns so we put that into the `id_vars`

In [59]:
am_data2 = pd.melt(am_data, id_vars=['id','section'], value_vars=['I1', 'I2', 'I3', 'I4'], 
                   var_name='question', value_name='correct')

In [61]:
a_data2.head()

,id,section,question,correct
0,200,afternoon,I1,0
1,201,afternoon,I1,1
2,202,afternoon,I1,1
3,203,afternoon,I1,0
4,204,afternoon,I1,0


In [51]:
?pd.melt

Signature:
pd.melt(
    frame: 'DataFrame',
    id_vars=None,
    value_vars=None,
    var_name=None,
    value_name: 'Hashable' = 'value',
    col_level=None,
    ignore_index: 'bool' = True,
) -> 'DataFrame'
Docstring:
Unpivot a DataFrame from wide to long format, optionally leaving identifiers set.

This function is useful to massage a DataFrame into a format where one
or more columns are identifier variables (`id_vars`), while all other
columns, considered measured variables (`value_vars`), are "unpivoted" to
the row axis, leaving just two non-identifier columns, 'variable' and
'value'.

Parameters
----------
id_vars : tuple, list, or ndarray, optional
    Column(s) to use as identifier variables.
value_vars : tuple, list, or ndarray, optional
    Column(s) to unpivot. If not specified, uses all columns that
    are not set as `id_vars`.
var_name : scalar
    Name to use for the 'variable' column. If None it uses
    ``frame.columns.name`` or 'variable'.
value_name : scalar, defaul